In [7]:
from math import sin, cos, log, ceil
import numpy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [12]:
g = 9.8
v_t = 4.9
C_D = 1/5
C_L = 1

v0 = v_t
x0 = 0
y0 = 10

In [13]:
def f(u):
    v = u[0]
    theta = u[1]
    x = u[2]
    y = u[3]
    return numpy.array([-g*sin(theta) - C_D/C_L*g/v_t**2*v**2,
                      -g*cos(theta)/v + g/v_t**2*v,
                      v*cos(theta),
                      v*sin(theta)])

In [14]:
def euler_step(u, f, dt):
    return u + dt * f(u)

In [15]:
T = 20
dt = 0.001
N = int(T/dt) + 1
t = numpy.linspace(0, T, N)
x_max = 0
theta_max = 0
ground = 0
u = numpy.empty((N,4))


for theta0 in range(0,90,):
    theta0 = theta0*numpy.pi/180
    u[0] = numpy.array([v0, theta0, x0, y0])

    for n in range(N-1):
        u[n+1] = euler_step(u[n], f, dt)
    x = u[:,2]
    y = u[:,3]
    
    y_ground = numpy.where(y<0)[0]
    if len(y_ground) == 0:
        print ("not touched ground yet")
    else:
        ground = y_ground[0]
    if x[ground] > x_max:
        x_max = x[ground]
        theta_max = theta0
        print (x_max, theta_max*180/numpy.pi)
    else:
        print (theta0 *180/numpy.pi)

50.4286038705 0.0
50.4429238114 1.0
50.4516361136 2.0
50.4547364916 3.0
50.4569803646 4.0
50.4583639313 5.0
6.0
7.0
8.0
9.0
10.0
10.999999999999998
12.0
13.000000000000002
14.0
14.999999999999998
16.0
17.0
18.0
19.0
20.0
21.0
21.999999999999996
23.0
24.0
25.0
26.000000000000004
27.0
28.0
29.0
29.999999999999996
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
43.99999999999999
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.00000000000001
53.0
54.0
55.0
56.0
57.0
58.0
59.0
59.99999999999999
61.0
62.0
63.0
64.0
65.0
66.0
66.99999999999999
68.0
68.99999999999999
70.0
71.0


/Users/AAA/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:6: RuntimeWarning: overflow encountered in double_scalars


ValueError: math domain error

In [ ]:
pyplot.figure(figsize=(8,6))
pyplot.grid(True)
pyplot.xlabel(r'x', fontsize=18)
pyplot.ylabel(r'y', fontsize=18)
pyplot.title('Glider trajectory, flight time = %.2f' % T, fontsize=18)
pyplot.plot(x,y, 'k-', lw=2);

In [ ]:
dt_values = numpy.array([0.1, 0.05, 0.01, 0.005, 0.001])

u_values = numpy.empty_like(dt_values, dtype=numpy.ndarray)

for i, dt in enumerate(dt_values):
    
    N = int(T/dt) + 1    # number of time-steps
    
    ### discretize the time t ###
    t = numpy.linspace(0.0, T, N)
    
    # initialize the array containing the solution for each time-step
    u = numpy.empty((N, 4))
    u[0] = numpy.array([v0, theta0, x0, y0])

    # time loop
    for n in range(N-1):
       
        u[n+1] = euler_step(u[n], f, dt)   ### call euler_step() ###
    
    # store the value of u related to one grid
    u_values[i] = u

In [ ]:
def get_diffgrid(u_current, u_fine, dt):
    N_current = len(u_current[:,0])
    N_fine = len(u_fine[:,0])
   
    grid_size_ratio = ceil(N_fine/N_current)
    
    diffgrid = dt * numpy.sum( numpy.abs(\
            u_current[:,2]- u_fine[::grid_size_ratio,2])) 
    
    return diffgrid

In [ ]:
# compute difference between one grid solution and the finest one
diffgrid = numpy.empty_like(dt_values)

for i, dt in enumerate(dt_values):
    print('dt = {}'.format(dt))

    ### call the function get_diffgrid() ###
    diffgrid[i] = get_diffgrid(u_values[i], u_values[-1], dt)

In [ ]:
# log-log plot of the grid differences
pyplot.figure(figsize=(6,6))
pyplot.grid(True)
pyplot.xlabel('$\Delta t$', fontsize=18)
pyplot.ylabel('$L_1$-norm of the grid differences', fontsize=18)
pyplot.axis('equal')
pyplot.loglog(dt_values[:-1], diffgrid[:-1], color='k', ls='-', lw=2, marker='o');

In [ ]:
r = 2
h = 0.001

dt_values2 = numpy.array([h, r*h, r**2*h])

u_values2 = numpy.empty_like(dt_values2, dtype=numpy.ndarray)

diffgrid2 = numpy.empty(2)

for i, dt in enumerate(dt_values2):
    
    N = int(T/dt) + 1   # number of time-steps
    
    ### discretize the time t ###
    t = numpy.linspace(0.0, T, N)
    
    # initialize the array containing the solution for each time-step
    u = numpy.empty((N, 4))
    u[0] = numpy.array([v0, theta0, x0, y0])

    # time loop
    for n in range(N-1):

        u[n+1] = euler_step(u[n], f, dt)         ### call euler_step() ###
    
    # store the value of u related to one grid
    u_values2[i] = u
    

#calculate f2 - f1
diffgrid2[0] = get_diffgrid(u_values2[1], u_values2[0], dt_values2[1])

#calculate f3 - f2
diffgrid2[1] = get_diffgrid(u_values2[2], u_values2[1], dt_values2[2])

# calculate the order of convergence
p = (log(diffgrid2[1]) - log(diffgrid2[0])) / log(r)

print('The order of convergence is p = {:.3f}'.format(p));